In [37]:
pickle_name = f"sparse_embedding.bin"
tfidfv_name = f"tfidv.bin"
emd_path = os.path.join(data_path, pickle_name)
tfidfv_path = os.path.join(data_path, tfidfv_name)

if os.path.isfile(emd_path) and os.path.isfile(tfidfv_path):
    with open(emd_path, "rb") as file:
        p_embedding = pickle.load(file)
    with open(tfidfv_path, "rb") as file:
        tfidfv = pickle.load(file)
    print("Embedding pickle load.")

Embedding pickle load.


In [200]:
import json
import os
import pickle
import time
from contextlib import contextmanager
from typing import List, NoReturn, Optional, Tuple, Union

import faiss
import numpy as np
import pandas as pd
from datasets import Dataset, concatenate_datasets, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)

In [38]:
import os

data_path = "../../data"
context_path = "wikipedia_documents.json"

In [205]:
data_path = "../../data"
context_path = "wikipedia_documents.json"
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

contexts = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)

In [39]:
a = pd.read_csv("./checkcheck.csv")
a = list(a['question'])
query = a

In [235]:
query

['처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 '스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?',
 '촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?',
 '로타이르가 백조를 구하기 위해 사용한 것은?',
 '의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?',
 '1945년 쇼와천황의 항복 선언이 발표된 라디오 방송은?',
 '징금수는 서양 자수의 어떤 기법과 같은 기술을 사용하는가?',
 '다른 과 의사들은 감염내과 전문의들로부터 어떤 것에 대해 조언을 받는가?',
 '루이 14세의 왕비 마리아 테래사는 어느 나라 공주인가?',
 '헤자즈 왕국이 실존했던 것은 언제까지인가?',
 '버드 교장이 5월의 여왕의 대안으로 제시한 것은?',
 "인형사'를 만들어낸 것으로 추측되는 사업의 이름은?",
 '멘데스가 요원들을 구하기 위해 간 도시는 어디인가?',
 '교과부의 행동에 화가나 여러명이 사직한 기구의 이름은?',
 '반대동맹이 공산당과 갈라서겠다고 얘기한 날은 언제인가?',
 '피어슨이 다시 의회를 해산했던 년도는?',
 '몽케가 죽은 뒤 쿠릴타이에서 대칸의 지위를 얻은 사람의 이름은?',
 '이흥구의 사법시험 이야기를 기사로 작성한 곳은?',
 '남북조 시대에서 이이 씨가 전쟁이 발생했을 때, 생활했던 장소는?',
 '박지훈은 1라운드에서 몇 순위를 차지했는가?',
 '데메카론에는 무엇을 풍자하는 이야기가 들어있나요?',
 '병에 걸려 죽을 확률이 약 25~50%에 달하는 유형의 질병은?',
 '설리반이 불만을 표시한 대상은 누구인가?',
 '베소스는 어디서 추방당했는가?',
 '진전사의 명칭이 드러나는 데 영향을 준 물건은?',
 '자신의 이상적인 국가관이 스파르타와 닮아 있다고 생각하는 플라톤의 저서는?',
 '박제된 북극곰이 사망한 날짜는?',
 '문법 측면에서 더 보수적인 포르투갈어 표준은?',
 '로스 수장이 살해한 사람은 어느 당 회원인가?',
 '조경숙왕의 아들인 요자의 친어머니는 누구

In [73]:
df = pd.DataFrame(p_embedding50[42016].T.todense(), index=tfidfv50.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(30))

            TF-IDF
##타 ##메   0.311063
스피 ##타    0.293771
##메 ##네스  0.261183
알렉산더      0.203378
##네스      0.193043
스피        0.164859
##메       0.145445
##네스 ##는  0.135804
베 ##소스    0.125861
마라 ##칸    0.122523
[UNK] n   0.121171
##칸 ##다   0.114480
n         0.104422
##타       0.098289
마케 ##도니아  0.098025
마케        0.096344
##도니아     0.094968
##네스 ##의  0.090353
알렉산더 ##에  0.083919
##그 ##디아  0.080390
알렉산더 ##가  0.079301
마라        0.079019
##디아 ##나  0.078804
알렉산더 ##는  0.077462
소 ##그     0.077057
박 ##트리    0.077057
##소스      0.073312
##노 ##케   0.072533
기원전 32    0.071855
##을       0.071200


In [19]:
import pandas as pd

In [120]:
query_vec = tfidfv.transform(query)
query_vec50 = tfidfv50.transform(query)
query_vec100 = tfidfv100.transform(query)

In [59]:
df = pd.DataFrame(query_vec[23].T.todense(), index=tfidfv.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(30))

            TF-IDF
##했 ##는   0.366545
##는 어디    0.350933
어디 ##서    0.336743
추방 ##당    0.327590
##소스      0.285313
##가 ?     0.264273
추방        0.262377
어디        0.249313
##는 ##가   0.241968
##당 ##했   0.226659
베         0.211548
?         0.202968
##당       0.139040
##는       0.094126
##서       0.080689
##했       0.077260
##가       0.054729
멘 ##셰     0.000000
멘 ##델     0.000000
멘         0.000000
멘토        0.000000
!         0.000000
멕시코 ,     0.000000
멕시코 ##의   0.000000
멕시코 ##에서  0.000000
멕시코 ##에   0.000000
멕시코 ##시티  0.000000
멕시코 ##로   0.000000
멕시코 ##는   0.000000
멕시코       0.000000


matrix([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])

In [46]:
import os

data_path = "../../data"
context_path = "wikipedia_documents.json"

pickle_name = f"sparse_embedding50.bin"
tfidfv_name = f"tfidv50.bin"
emd_path = os.path.join(data_path, pickle_name)
tfidfv_path = os.path.join(data_path, tfidfv_name)

if os.path.isfile(emd_path) and os.path.isfile(tfidfv_path):
    with open(emd_path, "rb") as file:
        p_embedding50 = pickle.load(file)
    with open(tfidfv_path, "rb") as file:
        tfidfv50 = pickle.load(file)
    print("Embedding pickle load.")

Embedding pickle load.


In [127]:
df = pd.DataFrame(query_vec100[5].T.todense(), index=tfidfv100.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(30))

            TF-IDF
##된 라디오   0.306103
##은 ?     0.296139
항복 선언     0.292185
##천 ##황   0.268171
##년 쇼     0.260912
선언 ##이    0.251132
##황 ##의   0.238515
방송 ##은    0.227953
##의 항복    0.217989
라디오 방송    0.215946
##이 발표    0.189323
쇼 ##와     0.186795
발표 ##된    0.175970
라디오       0.161107
항복        0.159471
1945 ##년  0.149145
1945      0.147900
##황       0.146625
방송        0.138695
선언        0.138612
쇼         0.137415
?         0.133109
##천       0.111445
발표        0.108255
##된       0.051218
##와       0.043413
##년       0.040732
##은       0.033266
##이       0.032634
##의       0.030593


In [67]:
cnt = 0
for i in query:
    if "했" in i:
        cnt+=1
        #print(i)
print(cnt)

19


In [118]:
import os

data_path = "../../data"
context_path = "wikipedia_documents.json"

pickle_name = f"sparse_embedding100.bin"
tfidfv_name = f"tfidv100.bin"
emd_path = os.path.join(data_path, pickle_name)
tfidfv_path = os.path.join(data_path, tfidfv_name)

if os.path.isfile(emd_path) and os.path.isfile(tfidfv_path):
    with open(emd_path, "rb") as file:
        p_embedding100 = pickle.load(file)
    with open(tfidfv_path, "rb") as file:
        tfidfv100 = pickle.load(file)
    print("Embedding pickle load.")

Embedding pickle load.


In [105]:
result = query_vec100 * p_embedding100.T
k = 3
doc_scores = []
doc_indices = []
if not isinstance(result, np.ndarray):
    print("!")
    result = result.toarray()
for i in range(result.shape[0]):
    sorted_result = np.argsort(result[i, :])[::-1]
    doc_scores.append(result[i, :][sorted_result].tolist()[:k])
    doc_indices.append(sorted_result.tolist()[:k])



!


In [85]:
a = pd.read_csv("./checkcheck.csv")

In [89]:
a= list(a['original_context'])

In [93]:
tmp = []
for v in a:
    for ii,vv in enumerate(contexts):
        if v == vv:
            tmp.append(ii)

In [97]:
contexts[47950]

'요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & 니피싱 철도의 설립 인가가 떨어졌을 때였다. 스카버러 남쪽과 코보콘크 마을을 잇는 철도 노선 공사가 시작되었고 1871년 6월에 억스브릿지까지 철도가 완공되었다. 이 노선은 이후 1871년 11월에 캐닝턴까지 연장되었고 1872년 11월에 코보콘크까지 완공되었다. \\n\\n이 노선의 철로 궤간은 1067mm로 협궤 노선이였다. 목재와 장작 수요가 선로 용량이 넘칠 정도로 많았지만 1870년대에 다른 철도와 마찬가지로 경제적 난황을 이기 못해 수익이 줄어들었다. 투자자들은 수익이 줄어든 이유로 철도가 협궤로 지어져서 다른 표준궤 노선과 연계하여 화물 운송을 할 수 없는 점을 들었다 \\n\\n1881년 7월, 미들랜드 철도가 토론토 & 니피싱 철도를 인수하였고 이 노선은 표준궤로 전환되었다. 미들랜드 철도의 영향은 스카버러에서도 볼 수 있는데 노선 동쪽을 따라 나란히 달리는 미들랜드 애비뉴가 이 철도 회사의 이름을 따서 지어졌다. 1884년, 미들랜드 철도는 그랜드 트렁크 철도에 인수되었고 이는 이후 캐나다 내셔널 철도에 다시 인수되었다. CN은 이 노선을 억스브릿지 선으로 명명하였다\\n\\n억스브릿지 선을 따라 달리는 여객 철도는 통근객을 위한 열차가 아니였다. 처음에는 론빌에 있는 미들랜드정션역과 토론토를 잇는 열차가 두 대씩 양방향으로 운행했고 한 대는 북동쪽으로 코보콘크까지 운행하였다. 토론토 북동쪽에 있던 시골 마을의 수요는 저조하기만 하였으며 20세기에 들어서서 도로 개량과 자동차 보편화로 쇠락의 길을 걷게 되었다\\n\\n1955년에는 코보콘크에서 억스브릿지까지 여객 열차 운행이 중단되었으며 30년 뒤 선로 또한 폐선되었다. 1963년에는 억스브릿지까지 이어지는 여객열차 운행이 중단되고 CN 열차는 토론토 유니언역에서 마컴으로 가는 열차를 5시 20분에 딱 한 대 운행하였다. 토론토로 돌아오는 열차는 존재하지 않았다'

In [107]:
ccnt = 0
for i, ii in zip(tmp, doc_indices):
    if i in ii:
     ccnt+=1

In [102]:
ccnt

230

In [108]:
acc = ccnt/240*100

In [109]:
a

65.41666666666667

In [124]:
    k =20
    result = query_vec100 * p_embedding100.T

    doc_scores = []
    doc_indices = []
    if not isinstance(result, np.ndarray):
        result = result.toarray()
    for i in range(result.shape[0]):
        sorted_result = np.argsort(result[i, :])[::-1]
        doc_scores.append(result[i, :][sorted_result].tolist()[:k])
        doc_indices.append(sorted_result.tolist()[:k])
    ccnt = 0
    indice = 0
    index = []
    for i, ii in zip(tmp, doc_indices):
        if i in ii:
         ccnt+=1
         index.append(indice)
        indice+=1
    print("top ",k," acc: ", ccnt/240*100)


top  20  acc:  87.08333333333333


In [255]:
tmp

[5294,
 47950,
 11891,
 55663,
 29185,
 23335,
 16171,
 54179,
 10001,
 16138,
 5983,
 5264,
 29979,
 9613,
 34822,
 52576,
 12748,
 53651,
 42566,
 25587,
 21562,
 4741,
 30932,
 42016,
 14233,
 9353,
 9855,
 44248,
 23856,
 34686,
 32363,
 12004,
 4792,
 49702,
 49502,
 47373,
 13194,
 4832,
 42712,
 51048,
 27704,
 40413,
 29834,
 30333,
 14801,
 15000,
 20988,
 5177,
 49540,
 33013,
 5287,
 15015,
 28253,
 23695,
 49489,
 34100,
 5068,
 41618,
 18935,
 5259,
 5269,
 4959,
 4804,
 5081,
 31254,
 29689,
 4963,
 42759,
 5705,
 4747,
 31555,
 20241,
 15653,
 8387,
 4663,
 29075,
 5224,
 7714,
 24486,
 46731,
 32778,
 16171,
 36572,
 27392,
 11676,
 55756,
 22337,
 55492,
 31963,
 31426,
 42467,
 21801,
 4651,
 53766,
 35358,
 48939,
 4583,
 25833,
 4846,
 5188,
 23291,
 42209,
 5256,
 23667,
 6925,
 23598,
 13676,
 23517,
 54652,
 21174,
 51565,
 4944,
 55273,
 16711,
 6258,
 29373,
 35175,
 8142,
 5159,
 8658,
 23623,
 55828,
 40862,
 21658,
 37140,
 4871,
 4834,
 25218,
 49488,
 5048

In [195]:
df = pd.DataFrame(query_vec1[5].T.todense(), index=tfidfv100.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(30))

         TF-IDF
병           1.0
!           0.0
멸망 ##후      0.0
멸망 ##전      0.0
멸망 ##하      0.0
멸망 ##하고     0.0
멸망 ##하기     0.0
멸망 ##하면     0.0
멸망 ##하여     0.0
멸망 ##한      0.0
멸망 ##한다     0.0
멸망 ##할      0.0
멸망 ##함      0.0
멸망 ##해      0.0
멸망 ##했      0.0
멸망 (        0.0
멸망 ##의      0.0
멸망 )        0.0
멸망 ,        0.0
멸망 .        0.0
멸망 뒤        0.0
멸망 등        0.0
멸망 때        0.0
멸망 받        0.0
멸망 시        0.0
멸망 시기       0.0
멸망 시키       0.0
멸망 이전       0.0
멸망 이후       0.0
멸망 ##이      0.0


In [134]:
df = pd.DataFrame(query_vec1[0].T.todense(), index=tfidfv100.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(30))

           TF-IDF
##는 바보   0.591400
입니다      0.493452
바보       0.432685
나 ##는    0.309117
?        0.269421
나        0.217924
##는      0.062472
멸망 직전    0.000000
멸망 ##하기  0.000000
멸망 ##하면  0.000000
멸망 ##하여  0.000000
멸망 ##한   0.000000
멸망 ##한다  0.000000
멸망 ##할   0.000000
멸망 ##함   0.000000
멸망 ##해   0.000000
멸망 ##했   0.000000
멸망 ##후   0.000000
멸망 (     0.000000
멸망 이전    0.000000
멸망 )     0.000000
멸망 ##하   0.000000
멸망 ,     0.000000
멸망 .     0.000000
멸망 뒤     0.000000
멸망 등     0.000000
멸망 때     0.000000
멸망 받     0.000000
멸망 시     0.000000
멸망 시기    0.000000


In [216]:
tfidfvvv = TfidfVectorizer(
            tokenizer=tokenizer.tokenize, ngram_range=(1, 2), max_features=1000000,
        )

In [201]:
tokenizer = AutoTokenizer.from_pretrained(
        "klue/bert-base",
        use_fast=True,
    )

In [233]:
context = ["나는 바보 입니다?", "나는 밥 인가요?","나는 용희입니닥?","나는 용희입니다?","나는 칽컭", "나는 ???"]
query_vec1 = tfidfvvv.transform(["나는 바보닥"])

In [231]:
aa = tfidfvvv.fit_transform(context)

In [238]:
tmp

[5294,
 47950,
 11891,
 55663,
 29185,
 23335,
 16171,
 54179,
 10001,
 16138,
 5983,
 5264,
 29979,
 9613,
 34822,
 52576,
 12748,
 53651,
 42566,
 25587,
 21562,
 4741,
 30932,
 42016,
 14233,
 9353,
 9855,
 44248,
 23856,
 34686,
 32363,
 12004,
 4792,
 49702,
 49502,
 47373,
 13194,
 4832,
 42712,
 51048,
 27704,
 40413,
 29834,
 30333,
 14801,
 15000,
 20988,
 5177,
 49540,
 33013,
 5287,
 15015,
 28253,
 23695,
 49489,
 34100,
 5068,
 41618,
 18935,
 5259,
 5269,
 4959,
 4804,
 5081,
 31254,
 29689,
 4963,
 42759,
 5705,
 4747,
 31555,
 20241,
 15653,
 8387,
 4663,
 29075,
 5224,
 7714,
 24486,
 46731,
 32778,
 16171,
 36572,
 27392,
 11676,
 55756,
 22337,
 55492,
 31963,
 31426,
 42467,
 21801,
 4651,
 53766,
 35358,
 48939,
 4583,
 25833,
 4846,
 5188,
 23291,
 42209,
 5256,
 23667,
 6925,
 23598,
 13676,
 23517,
 54652,
 21174,
 51565,
 4944,
 55273,
 16711,
 6258,
 29373,
 35175,
 8142,
 5159,
 8658,
 23623,
 55828,
 40862,
 21658,
 37140,
 4871,
 4834,
 25218,
 49488,
 5048

In [234]:
df = pd.DataFrame(query_vec1[0].T.todense(), index=tfidfvvv.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(30))

             TF-IDF
##닥        0.527696
바보         0.527696
##는 바보     0.527696
나          0.234244
나 ##는      0.234244
##는        0.234244
용 ##희      0.000000
용          0.000000
?          0.000000
밥 인가요      0.000000
밥          0.000000
바보 입니다     0.000000
인가요        0.000000
인가요 ?      0.000000
입니다        0.000000
[UNK]      0.000000
? ?        0.000000
##희 ##입니다  0.000000
##는 ?      0.000000
##희 ##입    0.000000
##희        0.000000
##입니다 ?    0.000000
##입니다      0.000000
##입 ##니    0.000000
##입        0.000000
##닥 ?      0.000000
##니 ##닥    0.000000
##니        0.000000
##는 용      0.000000
##는 밥      0.000000


In [237]:
query

['처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 '스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?',
 '촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?',
 '로타이르가 백조를 구하기 위해 사용한 것은?',
 '의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?',
 '1945년 쇼와천황의 항복 선언이 발표된 라디오 방송은?',
 '징금수는 서양 자수의 어떤 기법과 같은 기술을 사용하는가?',
 '다른 과 의사들은 감염내과 전문의들로부터 어떤 것에 대해 조언을 받는가?',
 '루이 14세의 왕비 마리아 테래사는 어느 나라 공주인가?',
 '헤자즈 왕국이 실존했던 것은 언제까지인가?',
 '버드 교장이 5월의 여왕의 대안으로 제시한 것은?',
 "인형사'를 만들어낸 것으로 추측되는 사업의 이름은?",
 '멘데스가 요원들을 구하기 위해 간 도시는 어디인가?',
 '교과부의 행동에 화가나 여러명이 사직한 기구의 이름은?',
 '반대동맹이 공산당과 갈라서겠다고 얘기한 날은 언제인가?',
 '피어슨이 다시 의회를 해산했던 년도는?',
 '몽케가 죽은 뒤 쿠릴타이에서 대칸의 지위를 얻은 사람의 이름은?',
 '이흥구의 사법시험 이야기를 기사로 작성한 곳은?',
 '남북조 시대에서 이이 씨가 전쟁이 발생했을 때, 생활했던 장소는?',
 '박지훈은 1라운드에서 몇 순위를 차지했는가?',
 '데메카론에는 무엇을 풍자하는 이야기가 들어있나요?',
 '병에 걸려 죽을 확률이 약 25~50%에 달하는 유형의 질병은?',
 '설리반이 불만을 표시한 대상은 누구인가?',
 '베소스는 어디서 추방당했는가?',
 '진전사의 명칭이 드러나는 데 영향을 준 물건은?',
 '자신의 이상적인 국가관이 스파르타와 닮아 있다고 생각하는 플라톤의 저서는?',
 '박제된 북극곰이 사망한 날짜는?',
 '문법 측면에서 더 보수적인 포르투갈어 표준은?',
 '로스 수장이 살해한 사람은 어느 당 회원인가?',
 '조경숙왕의 아들인 요자의 친어머니는 누구

In [254]:
leng = []
for i, v in enumerate(contexts):
    if len(v) >10000:
        print(i)
        print("")
        print(v)
        print("")

578

;성도(청두)시 成都市

성도,간칭으로 “융”(蓉)이라고 도 한다. 아시에서 처음으로 국제“미식의 도시”(美食之都)로 이름이 났고 고대로 부터”천부지국”(天府之国)이라는 아름다운 명칭을 얻었다.. 이백(李白)의 시편 에는 “아홉개 하늘은 성도에서 나왔고 천만개의 문은 그림으로 들어 오너라” (九天开出一成都，万户千门入画图) 진절하게 금관성(锦官城)의 부유와 수려함을 묘사 하였다. 사천의 전통극은 절기하고 풍부하며 례로 타거(托举), 개혜안(开慧眼), 얼굴 변하기(变脸), 불뿜기(喷火), 칼숨기기(藏刀)등은 사람들로 하여금 감탄을 끓이지 않 았다. 성도는 2010년 2월 28일에 “미식의 도시”로 칭호를 수여 받았다. 성도의 유명한 소식(著名小吃)은 마퍼 두부(麻婆豆腐), 쌍류 토끼 두(双流兔头), 부처폐편(夫妻肺片), 단단면(担担面), 용 초수(龙抄手), 종 물 만두(钟水饺), 한 만두 떡(韩包子), 삼대 포(三大炮), 뢰탕원(赖汤圆)등이다. 성도의 가장 좋은 여행 시간은 3월, 4월, 5월, 6월, 9월, 10월, 11월이며 청성산(青城山), 서령 설산(西岭雪山), 구룡구(九龙沟), 구봉산(九峰山), 천대산(天台山), 용지 산림공원(龙池森林公园) 등은 모두 성도의 가장 이상적인 피서 명승지이다. 성도에서 북쪽으로 올라 가다 보면 도강언(都江堰), 구채구(九寨沟)로 도달할 수 있고 서쪽의 천장 도로(川藏公路)를 따라 올라 가면 라싸(拉萨) 로 통하며 촬영 성지인 경로에는 신도교(新都桥) 해라구 빙천(海螺沟冰川), 임지(林芝) 등지 망강(芒康)에서 남쪽으로 내려가다 보면 매리설산(梅里雪山), 샹그리라(香格里亚), 려강(丽江)등 주요 여행 경치 지역이 나타 난다.

;덕양시 德阳市
덕양은 역사가 유구하고 파촉문화 발원지 중의 하나이며 여행자원은 하늘의 두터운 특혜를 가져 독특하다. 국내외 에서도 유명한 국가 중점 문화물 보호 단위(国家重点文物保护单位)이며 국가 처음으로의4A 급 풍경 명승지의 고대 촉국의 삼성퇴(三星堆)유적지; 보존이 완호하며 건축이 정밀하

In [245]:
leng.sort(reverse=True)

In [246]:
leng

[46099,
 41411,
 29016,
 27406,
 25351,
 25327,
 21701,
 19191,
 16077,
 15069,
 14309,
 13798,
 12517,
 12208,
 12079,
 12061,
 11795,
 11619,
 11415,
 10950,
 10811,
 10749,
 10112,
 10098,
 10091,
 9514,
 9313,
 9293,
 9227,
 9200,
 9090,
 9030,
 8618,
 8601,
 8531,
 8384,
 8378,
 8333,
 8329,
 8175,
 8092,
 8045,
 7949,
 7852,
 7827,
 7562,
 7553,
 7545,
 7536,
 7357,
 7229,
 7220,
 7155,
 7154,
 7141,
 7016,
 7008,
 6934,
 6849,
 6776,
 6753,
 6734,
 6710,
 6709,
 6654,
 6587,
 6577,
 6557,
 6494,
 6354,
 6354,
 6316,
 6293,
 6280,
 6277,
 6222,
 6211,
 6208,
 6206,
 6204,
 6177,
 6166,
 6161,
 6157,
 6151,
 6063,
 6055,
 6050,
 6042,
 5980,
 5937,
 5919,
 5916,
 5899,
 5780,
 5722,
 5713,
 5640,
 5633,
 5613,
 5592,
 5516,
 5515,
 5514,
 5483,
 5453,
 5429,
 5419,
 5405,
 5343,
 5339,
 5316,
 5300,
 5299,
 5284,
 5277,
 5265,
 5256,
 5237,
 5236,
 5225,
 5214,
 5199,
 5183,
 5177,
 5162,
 5160,
 5133,
 5125,
 5103,
 5085,
 5068,
 5060,
 5056,
 5050,
 5045,
 5027,
 5026,
 5025,
 5